
# Multiclass Classification

Building a neural network to solve a multiclass classification exercise using the PyTorch framework.<br>
Classification is one of the basic machine learning exercises. A trained model aims to predict the class of an input unit with high accuracy. <br>
This neural network uses supervised learning, meaning that the input datasets also provide target labels to train the model with. <br>
<br>
This Notebook has been generated automatically using the JupyterLab extension ***MLProvCodeGen***.
<br>

Original Source Code and inspiration from this article https://janakiev.com/blog/pytorch-iris/ <br>
Original author: N. Janakiev https://github.com/njanakiev Twitter: https://twitter.com/njanakiev
        

### Installs
Install required packages before running

In [ ]:
#pip install numpy===1.22.2 pandas===1.3.3 matplotlib===3.5.1 sklearn==0.0 torch===1.8.0 tqdm===4.60.0 ipywidgets===7.6.5 pytorch-model-summary===0.1.2 ipython===7.31.1 gputil===1.4.0 psutil===5.9.0 py-cpuinfo===8.0.0 prov===2.0.0 pydot===1.4.2 --user
#torch currently not supported with python 3.10, downgrading to python 3.9.7 possibly required

### Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import GPUtil
import psutil
import cpuinfo
import platform
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import roc_curve, auc, confusion_matrix, f1_score, mean_absolute_error, mean_squared_error 
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable
import tqdm
from tqdm import trange
import datetime
from datetime import date
from pytorch_model_summary import hierarchical_summary
import json
import ipywidgets as widgets
import webbrowser
import IPython
from IPython.display import display, Image
import prov
from prov.model import ProvDocument, Namespace, Literal, PROV, Identifier
from prov.dot import prov_to_dot
import os

plt.style.use('ggplot')

### Provenance Data 

In [ ]:
def get_size(bytes, suffix="B"):
    """
    Scale bytes to its proper format
    e.g:
        1253656 => '1.20MB'
        1253656678 => '1.17GB'
    """
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor

d1 = ProvDocument()
d1.add_namespace('prov', 'http://www.w3.org/ns/prov#')
d1.add_namespace('ex', 'https://github.com/TarekAlMustafa/MLProvCodeGen1.0/')
d1.add_namespace('foaf', 'http://xmlns.com/foaf/0.1/')
d1.add_namespace('p-plan', 'http://purl.org/net/p-plan')

e_MLProvCodeGen = d1.entity(
        'ex:MLProvCodeGen',(
            ('prov:type', PROV['Plan']),
))
ag_author = d1.agent(
        'ex:Tarek Al Mustafa',(
            ('prov:type', PROV['Person']),
            ('foaf:givenName', 'Tarek Al Mustafa'),
            ('foaf:mbox', '<tarek.almustafa@uni-jena.de>'),
            ('prov:role', 'Author'),
))
kernellist = !jupyter kernelspec list
e_notebook = d1.entity(
        'ex:notebook',(
            ('ex:programming_language','Python'),
            ('ex:programming_language_version', cpuinfo.get_cpu_info()['python_version']),
            ('ex:kernel','python3(ipykernel)'),
            ('prov:type', PROV['File']),
            ('ex:fileformat', '.ipynb'),
            ('ex:name', 'MulticlassClassification.ipynb'),
            ('ex:creation_date', str(date.today())),
            ('ex:last_modified', 'TODO'),
))
e_notebook.add_asserted_type('prov:Collection')
d1.wasAttributedTo(e_notebook, ag_author)
a_generateNotebook = d1.activity('ex:GenerateNotebook')
d1.wasAssociatedWith(a_generateNotebook, ag_author, plan=e_MLProvCodeGen)
d1.wasGeneratedBy(e_notebook, a_generateNotebook)

#set_experimentinfo
e_experimentinfo = d1.entity('ex:Cell Experiment Info', (
    ('ex:type', 'notebook_cell'),
    ('ex:type', 'p-plan:step'),
))
a_setexperimentinfo = d1.activity('ex:set_experiment_info()')
a_setdate = d1.activity('ex:date.today()')

d1.wasStartedBy(a_setexperimentinfo,e_experimentinfo, time=datetime.datetime.now())
d1.wasInformedBy(a_setexperimentinfo, a_setdate)  
d1.hadMember(e_notebook, e_experimentinfo)
e_experimentinfo_data = d1.entity(
    'ex:Experiment Info Data',(
        ('ex:title', 'Multiclass Classification'),
        ('ex:creation_date', str(date.today())),
        ('ex:task_type', 'MulticlassClassification'),
))
d1.wasGeneratedBy(e_experimentinfo_data, a_setexperimentinfo)

#set_hardware_info()
uname = platform.uname()
sysInfo = str(uname.system +' '+ uname.release +' Version: '+ uname.version +' Machine: '+ uname.machine)
    
svmem = psutil.virtual_memory()

GPUs = GPUtil.getGPUs()
gpuList = []
for gpu in GPUs:
    gpu_id = gpu.id
    gpu_name = gpu.name
    gpuList.append((gpu_id , gpu_name))

        
e_hardwareinfo = d1.entity('ex:Cell Hardware Info', (
    ('ex:type', 'notebook_cell'),
    ('ex:type', 'p-plan:step'),
))
a_sethardwareinfo = d1.activity('ex:set_hardware_info()')
a_platform_uname = d1.activity('ex:platform.uname()')
a_cpuinfo = d1.activity('ex:cpuinfo.get_cpu_info()')
a_svmemtotal = d1.activity('ex:svmem.total')
a_getsize = d1.activity('ex:get_size(svmem.total)')
a_GPUtilgetGPU = d1.activity('ex:GPUtil.getGPUs()')
d1.wasStartedBy(a_sethardwareinfo, e_hardwareinfo, time=datetime.datetime.now())
d1.wasInformedBy(a_sethardwareinfo, a_platform_uname)
d1.wasInformedBy(a_sethardwareinfo, a_cpuinfo)
d1.wasInformedBy(a_sethardwareinfo, a_svmemtotal)
d1.wasInformedBy(a_svmemtotal, a_getsize)
d1.wasInformedBy(a_sethardwareinfo, a_GPUtilgetGPU)
d1.hadMember(e_notebook, e_hardwareinfo)
e_hardwareinfo_data = d1.entity(
    'ex:Hardware Info Data',(
        ('ex:CPU', cpuinfo.get_cpu_info()['brand_raw']),
        ('ex:RAM',  get_size(svmem.total)),
        ('ex:Operating System', sysInfo),
        ('ex:GPUs', str(gpuList)),
))
d1.wasGeneratedBy(e_hardwareinfo_data, a_sethardwareinfo)

#set_packages
cpuInfo_version = !pip list | grep -i py-cpuinfo
pytorch_model_summary_version = !pip list | grep -i pytorch-model-summary


e_packages = d1.entity('ex:Cell Packages', (
    ('ex:type', 'notebook_cell'),
    ('ex:type', 'p-plan:step'),
))
a_setpackages = d1.activity('ex:set_packages()', )
a_getVersion = d1.activity('ex:{package_name}.__version__')
a_getVersion_py_cpuinfo = d1.activity('ex:!pip list | grep -i py-cpuinfo')
a_getVersion_pytorch_model_summary = d1.activity('ex:!pip list | grep -i pytorch-model-summary')
d1.wasStartedBy(a_setpackages, e_packages, time=datetime.datetime.now())
d1.wasInformedBy(a_setpackages,a_getVersion)
d1.wasInformedBy(a_setpackages,a_getVersion_py_cpuinfo)
d1.wasInformedBy(a_setpackages,a_getVersion_pytorch_model_summary)
d1.hadMember(e_notebook, e_packages)

e_packages_data = d1.entity(
    'ex:Packages Data',(
        ('ex:numpy', np.__version__),
        ('ex:pandas', pd.__version__),
        ('ex:matplotlib',  matplotlib.__version__),
        ('ex:sklearn', sklearn.__version__),
        ('ex:torch', torch.__version__),
        ('ex:tqdm', tqdm.__version__),
        ('ex:ipywidgets', widgets.__version__),
        ('ex:pytorch-model-summary', pytorch_model_summary_version[0]),
        ('ex:ipython', IPython.__version__),
        ('ex:gputil', GPUtil.__version__),
        ('ex:psutil', psutil.__version__),
        ('ex:py-cpuinfo', cpuInfo_version[0]),
        ('ex:prov', prov.__version__), 
))
d1.wasGeneratedBy(e_packages_data, a_setpackages)

### Data Ingestion

In [ ]:
#Data Ingestion
starttime = datetime.datetime.now()

iris = load_iris()
X = iris['data']
y = iris['target']
names = iris['target_names']
output_dim = len(names)
feature_names = iris['feature_names']
endtime = datetime.datetime.now()
executionTime = endtime-starttime

e_dataingestion = d1.entity('ex:Cell Data Ingestion', (
    ('ex:type', 'notebook_cell'),
    ('ex:type', 'p-plan:step'),
))


a_setdataingestion = d1.activity('ex:set_data_ingestion()', startTime=starttime, endTime=endtime, other_attributes={'prov:executionTime': str(executionTime)})
d1.wasStartedBy(a_setdataingestion, e_dataingestion)
d1.hadMember(e_notebook, e_dataingestion)
e_dataingestion_data = d1.entity(
    'ex:Data Ingestion Data',(
        ('ex:dataset_id', 'Iris'),
        ('ex:output_dimensions', output_dim),
        ('ex:samples_total',  len(X)),
        ('ex:feature_dimensions', len(feature_names)),
))
d1.wasGeneratedBy(e_dataingestion_data, a_setdataingestion)

### Data Preparation

In [ ]:
#Data Preperation
starttime = datetime.datetime.now()
# Scale data to have mean 0 and variance 1 
# which is importance for convergence of the neural network
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

endtime = datetime.datetime.now()
executionTime = endtime-starttime

e_datapreparation = d1.entity('ex:Cell Data Preparation', (
    ('ex:type', 'notebook_cell'),
    ('ex:type', 'p-plan:step'),
))
a_setdatapreparation = d1.activity('ex:set_data_preparation()', startTime=starttime, endTime=endtime, other_attributes={'prov:executionTime': str(executionTime)})

d1.wasStartedBy(a_setdatapreparation, e_datapreparation)
d1.used(a_setdatapreparation, e_dataingestion_data)
d1.hadMember(e_notebook, e_datapreparation)
e_datapreparation_data = d1.entity(
    'ex:Data Preparation Data',(
        ('ex:number_operations', 1),
        ('ex:transformation', 'sklearn.preprocessing.StandardScaler'),
        ('ex:transformation_method', 'Standardscaler.fit_transform'),  
))
d1.wasGeneratedBy(e_datapreparation_data, a_setdatapreparation)
d1.wasInfluencedBy(e_datapreparation, e_dataingestion_data)

### Data Segregation

In [ ]:
# Data Segregation
starttime = datetime.datetime.now()

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=2)

endtime = datetime.datetime.now()
executionTime = endtime-starttime

e_datasegregation = d1.entity('ex:Cell Data Segregation', (
    ('ex:type', 'notebook_cell'),
    ('ex:type', 'p-plan:step'),
))
a_setdatasegregation = d1.activity('ex:set_data_segregation()', startTime=starttime, endTime=endtime, other_attributes={'prov:executionTime': str(executionTime)})
d1.wasStartedBy(a_setdatasegregation, e_datasegregation)
d1.used(a_setdatasegregation, e_datapreparation_data)
d1.hadMember(e_notebook, e_datasegregation)
e_datasegregation_data = d1.entity(
    'ex:Data Segregation Data',(
        ('ex:segregation_method', 'sklearn.model_selection.train_test_split'),
        ('ex:test_size', 0.2),
        ('ex:train_size', 1-0.2), 
        ('ex:random_state', 2), 
))
d1.wasGeneratedBy(e_datasegregation_data, a_setdatasegregation)
d1.wasInfluencedBy(e_datasegregation, e_datapreparation_data)

### Data Visualization

In [ ]:
#Data Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
for target, target_name in enumerate(names):
    X_plot = X[y == target]
    ax1.plot(X_plot[:, 0], X_plot[:, 1], 
             linestyle='none', 
             marker='o', 
             label=target_name)
ax1.set_xlabel(feature_names[0])
ax1.set_ylabel(feature_names[1])
ax1.axis('equal')
ax1.legend();

for target, target_name in enumerate(names):
    X_plot = X[y == target]
    ax2.plot(X_plot[:, 2], X_plot[:, 3], 
             linestyle='none', 
             marker='o', 
             label=target_name)
ax2.set_xlabel(feature_names[2])
ax2.set_ylabel(feature_names[3])
ax2.axis('equal')
ax2.legend();


### Model

In [ ]:
starttime = datetime.datetime.now()

#Use GPU?
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

neuron_number = 50

#Configure Neural Network Models
class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_dim, 50)
        self.layer2 = nn.Linear(50, 50)
        self.layer3 = nn.Linear(50, output_dim)
        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.softmax(self.layer3(x), dim=1)
        return x

model     = Model(X_train.shape[1])
optimizer = torch.optim.Adam(model.parameters())
loss_fn   = nn.CrossEntropyLoss()

try:
    lr
except NameError:
    default = 1
    lr = None 
else:
    default = 0

endtime = datetime.datetime.now()
executionTime = endtime-starttime

e_modelparameters = d1.entity('ex:Cell Model Parameters', (
    ('ex:type', 'notebook_cell'),
    ('ex:type', 'p-plan:step'),
))
a_setmodelparameters = d1.activity('ex:set_model_parameters()', startTime=starttime, endTime=endtime, other_attributes={'prov:executionTime': str(executionTime)})
d1.wasStartedBy(a_setmodelparameters, e_modelparameters)
d1.used(a_setmodelparameters, e_datasegregation_data)
d1.hadMember(e_notebook, e_modelparameters)
e_modelparameters_data = d1.entity(
    'ex:Model Parameters Data',(
        ('ex:gpu_enable', 1),
        ('ex:modelParameters', 'str(model)TODO'),
        ('ex:neuron_number', neuron_number), 
        ('ex:loss_function', 'nn.CrossEntropyLoss()'), 
        ('ex:optimizer', 'torch.optim.Adam('), 
        ('ex:optimizer_default_learning_rate', default), 
        ('ex:optimizer_learning_rate', str(lr)), 
        ('ex:activation_function', 'F.softmax(self.layer3(x), dim=1)'),  
))
d1.wasGeneratedBy(e_modelparameters_data, a_setmodelparameters)
d1.wasInfluencedBy(e_modelparameters, e_datasegregation_data)

model

### Training

In [ ]:
#Model Training
starttime = datetime.datetime.now()

EPOCHS  = 100
X_train = Variable(torch.from_numpy(X_train)).float()
y_train = Variable(torch.from_numpy(y_train)).long()
X_test  = Variable(torch.from_numpy(X_test)).float()
y_test  = Variable(torch.from_numpy(y_test)).long()

loss_list     = np.zeros((EPOCHS,))
accuracy_list = np.zeros((EPOCHS,))

for epoch in trange(EPOCHS):
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)
    loss_list[epoch] = loss.item()
    
    # Zero gradients
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    with torch.no_grad():
        y_pred = model(X_test)
        correct = (torch.argmax(y_pred, dim=1) == y_test).type(torch.FloatTensor)
        accuracy_list[epoch] = correct.mean()

endtime = datetime.datetime.now()
executionTime = endtime-starttime
#set_training

e_training = d1.entity('ex:Cell Training', (
    ('ex:type', 'notebook_cell'),
    ('ex:type', 'p-plan:step'),
))
a_settraining = d1.activity('ex:set_training()', startTime=starttime, endTime=endtime, other_attributes={'prov:executionTime': str(executionTime)})
d1.wasStartedBy(a_settraining, e_training)
d1.used(a_settraining, e_modelparameters_data)
d1.hadMember(e_notebook, e_training)
e_training_data = d1.entity(
    'ex:Training Data',(
        ('ex:epochs', EPOCHS),
        ('ex:numberOfParameters', hierarchical_summary(model, print_summary = False)[1]),  
))
d1.wasGeneratedBy(e_training_data, a_settraining)
d1.wasInfluencedBy(e_training, e_modelparameters_data)

### Evaluation

In [ ]:
starttime = datetime.datetime.now()
#Plot Accuracy and Loss from Training
fig, (ax1, ax2) = plt.subplots(2, figsize=(12, 6), sharex=True)

ax1.plot(accuracy_list)
ax1.set_ylabel("validation accuracy")
ax2.plot(loss_list)
ax2.set_ylabel("validation loss")
ax2.set_xlabel("epochs");

### Confusion Matrix

In [ ]:
confusionMatrix = confusion_matrix(y_test, torch.argmax(y_pred, dim=1))
confusionMatrix

### F1 Score

In [ ]:
F1 = f1_score(y_test, torch.argmax(y_pred, dim=1), average=None)
F1

### Mean Absolute Error & Mean Squared Error

In [ ]:
MAE = mean_absolute_error(y_test, torch.argmax(y_pred, dim=1), multioutput='uniform_average')
MSE = mean_squared_error(y_test, torch.argmax(y_pred, dim=1), multioutput='uniform_average')
print(MAE)
print(MSE)

### ROC

In [ ]:
#ROC Curve 
plt.figure(figsize=(10, 10))
plt.plot([0, 1], [0, 1], 'k--')

# One hot encoding
enc = OneHotEncoder()
Y_onehot = enc.fit_transform(y_test[:, np.newaxis]).toarray()

with torch.no_grad():
    y_pred = model(X_test).numpy()
    fpr, tpr, threshold = roc_curve(Y_onehot.ravel(), y_pred.ravel())
    
plt.plot(fpr, tpr, label='AUC = {:.3f}'.format(auc(fpr, tpr)))
AUC = '{:.5f}'.format(auc(fpr, tpr))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend();

endtime = datetime.datetime.now()
executionTime = endtime-starttime
#set_evaluation

e_evaluation = d1.entity('ex:Cell Evaluation', (
    ('ex:type', 'notebook_cell'),
    ('ex:type', 'p-plan:step'),
))
a_setevaluation = d1.activity('ex:set_evaluation()', startTime=starttime, endTime=endtime, other_attributes={'prov:executionTime': str(executionTime)})
d1.wasStartedBy(a_setevaluation, e_evaluation)
d1.used(a_setevaluation, e_training_data)
d1.hadMember(e_notebook, e_evaluation)
e_evaluation_data = d1.entity(
    'ex:Evaluation Data',(
        ('ex:Accuracy', accuracy_list[(len(accuracy_list)-1)]),
        ('ex:Loss', loss_list[(len(loss_list)-1)]),
        ('ex:Confusion Matrix', str(confusionMatrix)),
        ('ex:AUC', float(AUC)),
        ('ex:F1 Score', str(F1)),
        ('ex:Mean Absolute Error', MAE),
        ('ex:Mean Squared Error', MSE), 
))
d1.wasGeneratedBy(e_evaluation_data, a_setevaluation)
d1.wasInfluencedBy(e_evaluation, e_training_data)

### Generate Provenance Data

In [ ]:
#add visualization to PATH
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

#get time for filenames
timestring = datetime.datetime.now().strftime('%Y-%m-%d--%H-%M-%S')
ProvenanceNameImage = ('Provenance_MulticlassClassification_' + timestring + '.png')

dot = prov_to_dot(d1, direction='RL')
dot.write_png('../GeneratedProvenanceData/'+ProvenanceNameImage)

provenanceImage_open = widgets.Button(description = 'Open Image File')
display(provenanceImage_open)

def on_button_clicked(b):
    provenanceImage_open.on_click = webbrowser.open('http://localhost:8888/lab/tree/GeneratedProvenanceData/'+ProvenanceNameImage)

provenanceImage_open.on_click(on_button_clicked)
Image('../GeneratedProvenanceData/'+ProvenanceNameImage)

### Write Provenance Data

In [ ]:
ProvenanceName = ('Provenance_MulticlassClassification_' + timestring + '.json')

with open('../GeneratedProvenanceData/'+ProvenanceName, 'w') as prov_file:
    prov_file.write(d1.serialize(indent=2))

### Show Provenance Data

In [ ]:
provenance_open = widgets.Button(description = 'Open Provenance Data File')
display(provenance_open)

def on_button_clicked(b):
    provenance_open.on_click = webbrowser.open('http://localhost:8888/lab/tree/extension/GeneratedProvenanceData/'+ProvenanceName)

provenance_open.on_click(on_button_clicked)